In [26]:
### Part 1

In [3]:
import os 
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
HuggingFaceApi = os.getenv('HF_TOKEN')
groq_api_key = os.getenv("GROQ_API_KEY")
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEndpoint


engine = create_engine("sqlite:///AppleSales.db")
db = SQLDatabase(engine=engine)



llm = ChatGroq(model = "llama-3.3-70b-versatile",api_key=groq_api_key,
               temperature= 1, max_tokens=1024
)



from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
system_message = prompt_template.format(dialect="SQLite", top_k=5)

from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

c:\vs_code\Genai\genai_env\Lib\site-packages\langsmith\client.py:5499: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [28]:
## Part 2

In [1]:
### Create 2nd tools to retreive question answer from the pdf

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate


pdf_data = PyPDFDirectoryLoader(r'C:\vs_code\Genai\pdf_directory').load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(pdf_data)


from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

from langchain_community.vectorstores import FAISS
vector_databse = FAISS.from_documents(text_splitter, embeddings)
retriever = vector_databse.as_retriever()

from langchain.tools.retriever import create_retriever_tool


prompt = ChatPromptTemplate.from_template(
    """
    answer the questions based on the provided context only.
    Please provide the most accurate  response based on the question 
    also make sure to give answer in simple english language 
    <context>
    {context}
    <context>
    Question:{input}
    
    """
        )

retriever_tool = create_retriever_tool(
    retriever,
    "PDF SEARCH",
    "Search Any Kind Of Information That Is Given  In The PDF Directory ",
    document_prompt= prompt
)

c:\vs_code\Genai\genai_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
combined_tools = tools + [retriever_tool] 
combined_tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002155CEF7310>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002155CEF7310>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002155CEF7310>),
 QuerySQLCheckerTool(description='Use this tool to 

In [9]:
memory = MemorySaver()

agent_executor = create_react_agent(llm, combined_tools, messages_modifier= system_message,
                                    )

C:\Users\javar\AppData\Local\Temp\ipykernel_7808\1005626057.py:3: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(llm, combined_tools, messages_modifier= system_message,


In [6]:
question = "Which State Is Having A Highest Sales?"
config = {"configurable": {"thread_id": "abc123"}}


for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    config = config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which State Is Having A Highest Sales?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_5vem)
 Call ID: call_5vem
  Args:
  sql_db_schema (call_02ms)
 Call ID: call_02ms
  Args:
    table_names: sales
  sql_db_query_checker (call_wdkk)
 Call ID: call_wdkk
  Args:
    query: SELECT State, SUM(Sales) as Total_Sales FROM sales GROUP BY State ORDER BY Total_Sales DESC LIMIT 1
  sql_db_query (call_n9ah)
 Call ID: call_n9ah
  Args:
    query: SELECT State, SUM(Sales) as Total_Sales FROM sales GROUP BY State ORDER BY Total_Sales DESC LIMIT 1
================================= Tool Message =================================
Name: sql_db_query

Error: (sqlite3.OperationalError) no such table: sales
[SQL: SELECT State, SUM(Sales) as Total_Sales FROM sales GROUP BY State ORDER BY Total_Sales DESC LIMIT 1]
(Background on this error at: https://sqlal

In [10]:

question = "what does anukriti say to apj ?"

agent_executor.invoke(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    
    config = config
        )

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jjcwt41jfw5vq7tj4vw08avx` service tier `on_demand` on : Limit 100000, Used 100084, Requested 292. Please try again in 5m25.132s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}